In [114]:
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import poisson
from scipy.stats import t
from scipy.stats import ttest_1samp
from scipy.stats import chi2
from scipy.stats import chisquare
from scipy.special import comb
import pandas as pd
import math
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

def afhankelijke_productregel(kans_als_eersteWaarIs,kans_enkel ):
    result =  kans_enkel *kans_als_eersteWaarIs
    return result

def bayes(afhankelijkekans ,totaleKans):
    result = afhankelijkekans/totaleKans
    
    return result

def totale_kans(kansen : list):
    result = 0
    for x in kansen:
        result += x
    return result

# Herhaling

## Kansen

### Laplace
- verzameling met mogelijke uitkomsten: U
- elke uitkomst is even waarschijnlijk (uniforme verdeling)
– gebeurtenis = verzameling van gewenste uitkomsten: G
– kans dat gebeurtenis optreedt P(G) = #G / #U

In [115]:
U = 36
G = 3
'P(G)'; P = G/U 
print(P)
print(P*100)

0.08333333333333333
8.333333333333332


## Rekenen met kansen

### Kans van tegengestelde gebeurtenis
Als je de kans weet dat een gebeurtenis zich zal voltrekken, dan kan je heel gemakkelijk de kans bereken dat deze zich niet zal voltrekken: 
P(niet G) = 1-P(G)

In [116]:
U = 36
G = 3
P = 1 -(G/U)
print(P)
print(P*100)

0.9166666666666666
91.66666666666666


### De Somregel
Gebeurtenissen kunnen bestaan uit deelgebeurtenissen
Stel dat de deelgebeurtenissen Gi (i=1,..n) niet overlappen, het zijn uitsluitende deelgebeurtenissen
We vragen ons af wat de kans is dat 1 van deze
gebeurtenissen optreedt:
P(G) = P(G1 OF G2 OF ... OF Gn )
= P(G1 u G2 u ... u Gn )
= P(G1) + P(G2) + ... + P(Gn )

In [117]:
# Voorbeeld: boek kaarten, kies een kaart.
# Wat is de kans dat de kaart een aas of een 4 is?
Aas = 4/52
Vier = 4/52
# Kans aas of vier
Som = Aas + Vier
print(Som)
print(Som*100)

0.15384615384615385
15.384615384615385


### Inverse Somregel
Stel dat de deelgebeurtenissen overlappen (niet uitsluitend zijn)
Stel: enkel 2 deelgebeurtenissen
– P(G) = P(G1 U G2 ) = P(G 1 ) + P(G 2 ) - P(G1 n G2 )

In [118]:
# Voorbeeld: boek kaarten
# – kies 1 kaart
# – wat is de kans dat deze een aas is of een harten kaart?
#G = Gaas U Gharten maar doorsnede is niet leeg
Aas = 4
Harten = 52/4
Overlap = 1

InverseSom = (Aas + Harten - Overlap) / 52
print(InverseSom)

0.3076923076923077


### Productregel
Een gebeurtenis bestaat weer uit deelgebeurtenissen
Wat is de kans dat alle deelgebeurtenissen tegelijk optreden?
• Als de deelgebeurtenissen "onafhankelijk" zijn, dan geldt:
P(G) = P(G 1 EN G2 EN ... EN Gn )
= P(G1 n G2 n ... n Gn )
= P(G1) x P(G2) x ... x P(Gn )

In [119]:
# Voorbeeld: counter-strike
#– 3 spelers
#• speler1 schiet 1 keer op 5 raak
#• speler2 schiet 1 keer op 4 raak
#• speler3 schiet 1 keer op 3 raak
#– 1 terrorist probeert door tunnel te geraken
#– spelers kunnen elk 1 keer schieten
#Wat is de kans dat de terrorist levend door de tunnel raakt?
#Dit is de kans dat speler1 mist (G1) EN speler2 mist (G2) EN speler3 mist (G3)
P = (4/5)*(3/4)*(2/3)
print(P)

0.4


### Afhankelijke Productregel
De gebeurtenissen kunnen ook afhankelijk zijn van elkaar.
– In dit geval is P(G1 n G2 n ... n Gn )
≠ P(G1) x P(G2) x ... x P(Gn )

Voorbeeld: groep studenten
• Kans dat iemand een meisje is, is 0,48
• Kans dat iemand een bril draagt is 0,2
• Wat is de kans dat een willekeurige persoon een
meisje met een bril is?
Probleem: misschien zijn alle brildragers jongens.
in dit geval is P(G) = 0 terwijl P(Gmeisje) x P(Gbril ) = 0,096
er is dus een afhankelijkheid tussen jongens en brildragers

Om de kans te berekenen dat een student een
brildragend meisje is, gebruiken we volgende formule:
• P(G meisje n G bril) = P(G bril | G meisje) x P(G meisje)

hierbij is P(Gbril | Gmeisje ) de kans dat de persoon een
bril draagt, gegeven dat het een meisje is (= een
voorwaardelijke kans).

In [120]:
# Als er dus geen meisjes zijn met bril, dan is dit 0
# • Stel dat 10% van de meisjes een bril draagt, dan is P(Gmeisje n Gbril) =
P = 0.1*0.48
print(P)

afhankelijke_productregel(0.48,0.1)

0.048


0.048

## Wet van de totale kans
Stel dat G1 , G2 , ..., Gn elkaar niet overlappen dan kan je volgende formule opstellen:


In [121]:
#Voorbeeld:
# – 3 politici hebben resp. 30%, 20% en 50% kans dat ze verkozen worden (V1, V2, V3) –als minister van financiën-
# – de kans dat de politici de belastingen verlagen is resp. 50%, 40% en 30%
# – wat is de kans dat de belastingen verlaagd worden?
# 𝑃(Bverlaagd) = P(Bverlaagd | V1) * (V1) + P(Bverlaagd | V2) * (V2) + P(Bverlaagd | V3) * (V3)
P = (0.50*0.30)+(0.40*0.20)+(0.30*0.50)
print(P)

totale_kans([afhankelijke_productregel(0.5,0.3),afhankelijke_productregel(0.4,0.2),afhankelijke_productregel(0.3,0.5)])

0.38


0.38

## Wet van Bayes
Stel dat {G1, G2, ..., Gn } een partitie is van G dan zegt de wet van Bayes:
P(Gk|B) = P(B|Gk)*P(Gk)/P(B)

Stel nu dat na de verkiezingen de belastingen verlaagd worden. Hoeveel kans is er dan dat dat komt doordat politicus 3 verkozen werd?
P(V3|Bverlaagd) = P(Bverlaagd|V3)*P(V3)/P(Bverlaagd)

In [147]:
#Voorbeeld
#Bob pendelt elke dag naar zijn werk
# • Kans om te laat te komen: – trein (10%), bus (20%), auto (40%)
# • Op zekere dag komt Bob te laat. Wat is de kans dat hij met de auto kwam?
# Stel: alle voertuigen met evenveel kans gebruikt
# P(auto|telaat) = P(Bverlaagd|V3)*P(V3)/P(Bverlaagd)
print(bayes(afhankelijke_productregel(0.40,0.1),totale_kans([(0.10*0.80),(0.20*0.1),(0.40*0.1)])))

0.28571428571428575


### Kruistabellen en kansen

|          | Wit merk | Geen Wit merk |        |
|:--------:|:--------:|:-------:|:------:|
| Slechte koeling |   1498  | 1513   |  3011   |
| Goede koeling   |  504    | 6485   | 6989   |
|                 |   2002    |    7998   |  10000   |

In [123]:
# Wat is de kans dat een pc van een wit merk een slechte koeling heeft?
# U = {PC's van wit merk}, G {Slechte koeling}
U = 1498
G = 2002
L = U/G
print(L)

0.7482517482517482


# Kansverdelingen
## Wat is een kansverdeling?
- doe een theoretische oneindige steekproef/experiment
– bepaal de relatieve frequenties van iedere waarde
– deze tabel noemen we "kansverdeling"
## Verband met frequentietabel
### Kansverdelingen en frequentietabellen
° Kansverdeling:
– theoretisch bepaald
– steekproef wordt oneindig groot verondersteld
° Frequentietabellen:
– experimenteel bepaald
– eindige steekproef
– benadert kansverdeling
• Allebei: relatieve frequenties

## Centrum- en spreidingsmaten van een kansverdeling
### Centrummaten en spreidingsmaten
• Kansen en relatieve frequenties zijn gelijkaardig
• Is er dan een gemiddelde, mediaan, modus (centrummaten kansverdeling)?
• Is er een IQR, variantie, standaardafwijking (spreidingsmaten kansverdeling)?

### Centrummaten kansverdeling
Gemiddelde noemen we "verwachte waarde"
Dit is namelijk de waarde die we verwachten voor de populatie
Symbool: µ
µ = som van (elke waarde * kans op waarde)

In [124]:
# Voorbeeld: multiple choice examen met giscorrectie
#– 1 vraag
#– keuze uit 4 antwoorden
#– juist antwoord = 1 punt
#– fout antwoord = -1/4 punt
#– kans op een goed antwoord is p
#Stel dat je er niks van kent. Wat is p dan? Wat is dan de verwachte score?
p = 1/4
µ = 1*1/4 - 1/4*3/4
print(µ)
#Stel dat je twijfelt tussen 3 antwoorden. Wat is p? Wat is dan de verwachte score?
p = 1/3
µ = 1*1/3- 1/4*2/3
print(µ)
#Stel dat je twijfelt tussen 2 antwoorden. Wat is p? Wat is dan de verwachte score?
p = 1/2
µ = 1*1/2- 1/4*1/2
print(µ)
#Is het nu beter om te gokken of niks in te vullen?

0.0625
0.16666666666666666
0.375


### Spreidingsmaten kansverdeling
Variantie en standaardafwijking zijn ook verondersteld op de hele populatie
We gebruiken dus symbolen σ² en σ
Voor het gooien met 2 dobbelstenen is dat:
σ² = (2-7)²*1/36+...+(7-7)²*1/36+...+(12-7)²*1/36=5,8333
σ = V5,8333 = 2,415

## Veel voorkomende kansverdelingen
### De binomiale verdeling
Is een model voor een experiment waarbij:
– er maar 2 uitkomsten mogelijk zijn (‘succes’ en ‘geen succes’)
– iedere uitkomst een bepaalde waarschijnlijkheid heeft:
    eerste mogelijke uitkomst (succes) : p
    tweede mogelijke uitkomst (geen succes): 1-p
- experiment wordt verschillende achtereenvolgende keren (n) uitgevoerd en de resultaten worden gemeten
- de vraag is: “Hoeveel kans is er dat bij n experimenten er x gelijk zijn aan de eerste mogelijke uitkomst?“(maw kans dat x experimenten op n succesvol zijn?)

In [125]:
# De binomiale verdeling - Voorbeeld
# • Examen met 5 meerkeuzevragen
# • Iedere vraag heeft 4 mogelijke antwoorden, waarvan 1 juist
# • Wat is de kans dat we exact 2 vragen juist beantwoorden?
# – we hebben niet gestudeerd en gokken dus...
# – er is geen giscorrectie
# n = aantal vragen
# x = aantal juist
# p = kans op succes
# Wat is dus de kans om 0, 1, 2, 3, 4 en 5 vragen juist te hebben?
n = 5
x = range(0,6)
p = 1/4
print(binom.pmf(x,n,p)*100) #probability mass function
# Wat is dus de kans om te slagen op dit examen? P(x>=3) = P(x=3) + P(x=4) + P(x=5) = 1 – P(x<=2)
binom.pmf(3,5,1/4) + binom.pmf(4,5,1/4) + binom.pmf(5,5,1/4)
1-(binom.pmf(0,5,1/4) + binom.pmf(1,5,1/4) + binom.pmf(2,5,1/4))
1 - binom.cdf(2,5,1/4) #cumulative density function

[23.73046875 39.55078125 26.3671875   8.7890625   1.46484375  0.09765625]


0.103515625

### Verwachte waarde
Verwachte waarde = hoeveel vragen zullen gemiddeld goed zijn?
μ = n * p
In dit geval: 5 * 1/4 = 1,25

In [126]:
binom.mean(5,1/4)

1.25

### Standaardafwijking
De standaardafwijking is ook gemakkelijk te berekenen: σ Vn*p*(1-p)
De standaardafwijking is hier dus: sqrt(5*1/4*3/4) = 0,968

In [127]:
binom.std(5,1/4)

0.9682458365518543

In [128]:
#Examen computersystemen:
#– 20 multiple choice vragen
#– 4 keuzemogelijkheden per vraag
#• Wat is de verwachte score als iedereen gokt?
#– het gemiddelde was 7/20...
#• Wat is de standaardafwijking?
binom.std(20, 1/4)

1.9364916731037085

### De normale verdeling
Is een model voor een experiment waarbij:
– de variabele continu is
– de kansen symmetrisch verdeeld zijn (evenveel kans om meer of minder dan het gemiddelde uit te komen)
– de verwachtingswaarde en verwachte standaardafwijking gekend zijn
formule in python: norm.pdf(x, loc=μ, scale=σ)
#### Berekenen van de oppervlakte
• Grafiek is bepaald door μ en σ
• Hoe berekenen we de oppervlakte?
– integraal uitrekenen...
– tabel (tabel voor iedere combinatie van μ en σ?)
– gebruik de cumulatieve kansverdeling
in Python:
> norm.cdf (190, loc=180, scale=10) - norm.cdf(170, loc=180, scale=10)
Kans dat x tussen x1 en x2 ligt = oppervlakte tussen x1 en x2

In [129]:
# voorbeeld: kans dat lengte tussen 175 en 180 ligt:
U=(norm.cdf(180,loc=180,scale=10)-norm.cdf(175,loc=180,scale=10))
print(U)

0.19146246127401312


### De standaardnormale verdeling
• Verwachte waarde = 0
• Standaardafwijking = 1
• Je kan iedere normale verdeling omzetten in de standaardnormale via een "Z-transformatie" (cfr Z-scores): Z = (X-μ)/σ
• Kan gebruikt worden om oppervlaktes te bepalen adhv de tabel van de standaardnormale verdeling

### De Poisson verdeling
Is een model voor een experiment waarbij:
– de variabele discreet is
– de kansen weergeven hoeveel keer bepaalde voorvallen gedurende een gegeven tijdsinterval, afstand, gebied, volume,... voorkomen
– De parameter λ (een positief getal) het verwachte aantal voorvallen in het tijdsinterval weergeeft.
λ is eveneens het gemiddelde van de Poisson verdeling.

in Python:
from scipy.stats import poisson
>>> poisson.pmf(k, λ)
>>> k=range(0,13)
>>> poisson.pmf(k, 3) # λ = 3
De som van alle ‘staafjes’ is gelijk aan 1

In [130]:
k=range(0,13)
poisson.pmf(k, 3) # λ = 3

array([4.97870684e-02, 1.49361205e-01, 2.24041808e-01, 2.24041808e-01,
       1.68031356e-01, 1.00818813e-01, 5.04094067e-02, 2.16040315e-02,
       8.10151179e-03, 2.70050393e-03, 8.10151179e-04, 2.20950322e-04,
       5.52375804e-05])

Cummulatieve verdelingsfunctie:
• in Python:
>>> poisson.cdf(2, 3)

In [131]:
#In België zijn er ongeveer 10.000 branden in gebouwen per jaar. Er zijn ongeveer 4.500.000 gebouwen. 
# In een kleine gemeente zijn er 9.000 gebouwen.
# a. Bereken de kans dat er 25 branden in 2023 in de gemeente zullen plaats vinden.
# b. Bereken ook de kans op minstens 25 branden in 2023.
print(poisson.pmf(25,20))
1 - poisson.cdf(24,20)

0.04458764910826764


0.15677262182623775

## Toetsen
### Betrouwbaarheidsintervallen

#### Normaalverdeling
Probleemstelling
“We willen op basis van een steekproef iets zeggen over de populatie”
 Betrouwbaarheidsinterval
Tussen welke grenzen, bekomen adhv de parameters van een steekproef, ligt een bepaalde parameter van de populatie? Hoe 'zeker' is deze uitspraak?

#### Betrouwbaarheidsinterval
Stel dat het verbruik van laptops normaal verdeeld is
 gemiddelde is μ
 standaardafwijking is σ
Kies een willekeurige 1 laptop
 Wat is de kans dat het verbruik tussen x en y ligt?
 Hoe zou je dit berekenen? >>> norm.cdf(y, loc= μ, scale= σ) ‐ norm.cdf(x, loc= μ, scale= σ)

Stel dat je meer dan 1 laptop willekeurig kiest (je kiest er n) maw je neemt een steekproef:
 Neem het gemiddelde verbruik van die n laptops
 Is er meer of minder kans om het echte gemiddelde μ uit te komen?
 Wat zegt dit over de standaardafwijking?

Stel dat je meerdere steekproeven neemt (telkens kies je aselect n laptops):
 Levert elke steekproef hetzelfde gemiddelde verbruik van n laptops op? Er werd toch gestart vanuit dezelfde populatie?
 Wat is de verdelingsfunctie voor het gemiddelde bekomen aan de hand van een steekproef?

“Centrale limietstelling” –zie simulatie-
 steekproef met grootte n (> 30)
 de kans om een bepaald gemiddelde uit te komen is bepaald door een normaalverdeling met als gemiddelde μ
 standaardafwijking σ/sqrt(n)
 dus 95,5% kans dat:
 en dus is er 95,5% kans dat:
 we benaderen σ door s:

In [132]:
#Met Python:
df_c = pd.read_csv('CSV/consumptionLaptops.csv', decimal='.')
x_bar = df_c.consumptionLaptops.mean()
s = df_c.consumptionLaptops.std()
n = len(df_c)
factor = norm.ppf((1+0.955)/2)
interval = (x_bar-factor*s/math.sqrt(n), x_bar+factor*s/math.sqrt(n))
#OR
norm.interval(confidence=0.955, loc=x_bar, scale=s/math.sqrt(n) )


FileNotFoundError: [Errno 2] No such file or directory: 'CSV/consumptionLaptops.csv'

#### Studenten-verdeling
Probleem: we benaderen σ door s
 Als n klein is (<30) is s een slechte benadering  normaalverdeling is niet goed
 Gebruik t-verdeling (student-verdeling)
 Een t-verdeling heeft een extra parameter: steeds gelijk aan n-1
 In Python: >>> from scipy.stats import t
>>> t.cdf(x,df, loc=μ, scale=s) #s as approx. of σ
>>> t.ppf(q,df, loc=μ, scale=s) #s as approx. of σ

In [133]:
#Met Python:
df_c = pd.read_csv('CSV/consumptionLaptops.csv', decimal='.')
x_bar = df_c.consumptionLaptops.mean()
s = df_c.consumptionLaptops.std()
n = len(df_c)
factor = t.ppf((1+0.955)/2, n-1, loc= x_bar, scale=s/math.sqrt(n))
interval = (x_bar-factor*s/math.sqrt(n), x_bar+factor*s/math.sqrt(n))
#OF:
t.interval(confidence=0.955, df=n-1, loc=x_bar, scale=s/math.sqrt(n))

FileNotFoundError: [Errno 2] No such file or directory: 'CSV/consumptionLaptops.csv'

### Toetsen van hypothesen
Probleemstelling: 
“Kunnen we een bewering over de populatie (= hypothese)weerleggen aan de hand van een steekproef?”

 Aanvaardingsinterval
Valt een steekproefparameter binnen bepaalde grenzen, bekomen adhv de bewering over de populatie? Hoe 'zeker' is deze uitspraak?

#### Hypothese
 Bij toetsen gaan we uit van een veronderstelling (mbt de populatie) en proberen we na te gaan (adhv een steekproef) of we die kunnen bevestigen of verwerpen
 Hypothese = Veronderstelling
 We proberen de hypothese tegen te spreken
     Als we een meting doen die in de lijn ligt van de veronderstelling, hebben we niks bewezen
     Als we een meting doen die de hypothese tegen-spreekt, is er veel kans dat deze niet waar is.

 2 hypothesen
 H 0: nul-hypothese
 H 1: alternatieve hypothese

 Een besluit is nooit 100% zeker

 Er is steeds een “significantieniveau” : σ
 de kans dat ons besluit niet juist is
 meestal is σ=0,05

 Als σ=0,05, dan is er 5% kans dat we een fout maken:
Fout bij het verwerpen van H0 terwijl die toch waar is (type I-fout): σ
 Maar er is ook een fout bij het aanvaarden van H0 terwijl die niet waar is (type II-fout): 


#### t-toets
 Voorbeeld
 H 0: laptops verbruiken gemiddeld 31 Watt H 0 : μ = 31
 H 1: laptops verbruiken gemiddeld meer of minder dan 31 Watt H 1 : μ =/ 31
 Werkwijze
 bepaal significantieniveau σ
 doe een steekproef
 stel een aanvaardingsinterval op (p=1-σ)
 trek een besluit

 In ons voorbeeld
 σ=0,05
 30 laptops, σ onbekend, dus gebruik t-verdeling
 aanvaardingsinterval (0,95)
>>> t.ppf(0.975,29) = 2,045
 23,53 = 31 – 2,045 . 20 / sqrt(30)
 38,46 = 31 + 2,045 . 20 / sqrt(30)
 We weten dus 95% zeker dat het gemiddelde verbruik tussen deze grenzen zou moeten liggen indien H0 waar is
 Dus...? (H 0 stelde dat μ gelijk is aan 31 Watt en x = 40 Watt)

In [ ]:
#Met Python:
df_c = pd.read_csv('CSV/consumptionLaptops.csv', decimal='.')
x_bar = df_c.consumptionLaptops.mean()
s = df_c.consumptionLaptops.std()
n = len(df_c)
mu0=31
factor = t.ppf((1+0.95)/2, n-1)
interval = (mu0-factor*s/math.sqrt(n), mu0+factor*s/math.sqrt(n))
#OF:
t.interval(confidence=0.95, df=n-1, loc=mu0, scale=s/math.sqrt(n))

Stel dat H 0 zei dat het gemiddelde verbruik
45 Watt is
 Welk besluit trek je nu?
t-toets
σ=0,05
 35,53 = 45 – 2,045 . 20 / sqrt(30)
 52,46 = 45 + 2,045 . 20 / sqrt(30)

In [134]:
t.interval(confidence=0.95, df=30-1, loc=45,scale=20/math.sqrt(30))

(37.531877264838, 52.468122735162)

#### Z-toets
Soms weet je de waarde van σ wel.
 Gebruik dan de normaalverdeling voor het aanvaardingsinterval

### Grenswaarde σ bepalen  p-waarde
#### Grenswaarde σ bepalen
in Python, voer een ‘one sample t-test’ uit:
from scipy.stats import ttest_1samp
screens = pd.read_csv('screens.csv',delimiter=';', decimal='.')
mu=40
ttest_1samp(screens["New_size"],mu)

In [135]:
screens = pd.read_csv('CSV/screens.csv',delimiter=';', decimal='.')
mu=40
ttest_1samp(screens["New_size"],mu)

FileNotFoundError: [Errno 2] No such file or directory: 'CSV/screens.csv'

#### Chi-Kwadraat toets
• Stel dat je wil nagaan of een variabele een bepaalde verdeling heeft
• Voorbeeld:
– Eigenaar van een winkel verkoopt computers, films, spelletjes en televisies
– Hij denkt dat de verkoop als volgt verdeeld is
> computers: 30%
> films: 10%
> spelletjes: 30%
> televisies: 30%

– Vraag: klopt dit?

• We doen een steekproef (30 klanten):
> computers: 15
> films: 3
> spelletjes: 7
> televisies: 5

• We hadden verwacht:
> computers: 9
> films: 3
> spelletjes: 9
> televisies: 9

• Vraag: hoe “ver” liggen deze van elkaar?

Dit is een soort afstand tussen de geobserveerde
frequenties en de verwachtte frequenties
• Frequenties zijn altijd absolute frequenties!
• Als H0 waar is, dan is er een grote kans om een kleine X² te vinden
• als X² groot is, dan is H0 dus waarschijnlijk fout
• in ons geval:X² = (15-9)²/9+(3-3)²/3+(7-9)²/9+(5-9)²/9

• De chi-kwadraat verdeling wordt bepaald door een parameter m-1
• m is het aantal waarden die geteld werden (4 in ons geval)
• We kunnen nu weer een 95% aanvaardingsinterval opstellen (m.a.w. σ = 0.05)
> begint altijd van 0
> gaat tot een grenswaarde:
>> from scipy.stats import chi2
>> chi2.ppf ((1- σ), df=m-1)

In [136]:
chi2.ppf ((1-0.05),df=4-1)

7.814727903251179

• Je kan de test ook als volgt doen:
> from scipy.stats import chisquare
> measured_values = [15,3,7,5]
> expected_values = [9,3,9,9]
> chisquare(measured_values , expected_values)

In [137]:
measured_values = [15,3,7,5]
expected_values = [9,3,9,9]
chisquare(measured_values, expected_values)

Power_divergenceResult(statistic=6.222222222222222, pvalue=0.10128520246221875)

## Association rules
### Inleiding Association rules
Het idee komt vanuit Market Basket Analysis (MBA)
 Wat koopt mijn klant? Welke producten worden er samen gekocht?
 Doel: associaties vinden tussen de verschillende producten dat een klant in zijn winkelkarretje plaatst

### Transacties en itemsets
#### Understanding the data
 Verschillende ‘manieren’ om data te verzamelen/weer te geven
 Elke ‘manier’ geeft een lijst weer van items die op hetzelfde moment werden aangekocht.

We moeten de input omzetten naar het juiste formaat
• In Python
> receipts = pd.read_csv('transactions.csv', sep=';', index_col=0)
> receipts_dum = pd.get_dummies(receipts, columns=['Product'], prefix='',prefix_sep='', dtype=bool)
> transactionstable = receipts_dum.groupby(level='TransactionID').max()

In [138]:
receipts = pd.read_csv('CSV/transactions.csv', sep=';', index_col=0)
receipts_dum = pd.get_dummies(receipts, columns=['Product'], prefix='',prefix_sep='', dtype=bool)
transactionstable = receipts_dum.groupby(level='TransactionID').max()
print(transactionstable)

FileNotFoundError: [Errno 2] No such file or directory: 'CSV/transactions.csv'

### Hoe doe je zo'n dataset
[01] {Printer,Cartridge,Ballpoint} 1000123
[02] {Paper,Cartridge} 1000124
[03] {Paper,Cartridge} 1000125
[04] {Printer,Cartridge} 1000126
[05] {Printer, Cartridge} 1000127
[06] {Ballpoint} 1000128
[07] {Paper,Cartridge,Ballpoint} 1000129
[08] {Cartridge} 1000130
[09] {Printer,Ballpoint} 1000131
[10] {Paper} 1000132

In [139]:
trans_itemlist = pd.read_csv('CSV/transactions_itemlist.csv', sep=';')
def get_item_list (string): 
    items = string [1:-1]
    return items.split(',')
transactions = trans_itemlist['items'].apply(get_item_list).to_numpy()
#transactions = trans_itemlist['items'].apply(lambda row:row[1:-1].split(';')).to_numpy()
te = TransactionEncoder()
dataset = te.fit(transactions).transform(transactions)
transactionstable = pd.DataFrame(dataset, columns=te.columns_, index= trans_itemlist.transactionID)
print(transactionstable)

FileNotFoundError: [Errno 2] No such file or directory: 'CSV/transactions_itemlist.csv'

### Modeling > Support
Support: Hoe vaak komt een combinatie van items voor?

Support (A) = #A/#Transacties
Support (A&B&...&M) = #(A&B&...&M)/ #Transacties

• Een combinatie van 1 item is ook een combinatie
• Support(Item) : Aantal keer dat item voorkomt relatief ten opzichte van het aantal transacties

Printer wordt verkocht bij 4 sales: De support van printer is 4/10 = 0.4
 Support(Printer) = 0,4

• Support voor een combinatie van meerdere items (itemset)
• Support(item1, item2): Relatief aantal keer dat item1 en item2 samen in een transactie voorkomen.
Support(printer EN cartridge) = 3/10 = 0,3

### Itemsets = Combinatie van items: 
Beschouw een winkel met slechts 4 producten:
from scipy.special import comb
comb(4,1) # = 4
comb(4,2) # = 6
comb(4,3) # = 4
comb(4,4) # = 1
sum([comb(4,i) for i in range(1,5)]) # = 15

In [ ]:
comb(4,1) # = 4
comb(4,2) # = 6
comb(4,3) # = 4
comb(4,4) # = 1
sum([comb(4,i) for i in range(1,5)]) # = 15

In [140]:
# Oefening:
# • Bereken hoeveel itemsets je kan maken wanneer de winkel 1000 producten (items) te koop aanbiedt.
# Association rules – Itemsets Item sets = Combinatie van items Oefening:
print(sum([comb(1000,i) for i in range(1,1001)]))

# • Bereken hoeveel producten je in een winkel maximaal kunt aanbieden waarbij het aantal itemsets niet hoger is dan het aantal atomen in het heelal (nl. 10^80 )
for j in range(250,275): 
    print (j, " ", sum([comb(j,i) for i in range(1,j+1)]))

1.0715086071867646e+301
250   1.8092513943332126e+75
251   3.6185027886659254e+75
252   7.237005577332468e+75
253   1.447401115466327e+76
254   2.8948022309327808e+76
255   5.789604461866955e+76
256   1.1579208923731074e+77
257   2.3158417847461918e+77
258   4.631683569492936e+77
259   9.263367138988058e+77
260   1.8526734277969404e+78
261   3.705346855594189e+78
262   7.410693711187493e+78
263   1.4821387422377835e+79
264   2.9642774844751363e+79
265   5.92855496895105e+79
266   1.185710993790014e+80
267   2.3714219875802067e+80
268   4.74284397516071e+80
269   9.485687950319638e+80
270   1.8971375900643453e+81
271   3.794275180128938e+81
272   7.588550360257002e+81
273   1.5177100720517926e+82
274   3.035420144102721e+82


 we hebben efficiëntere algorithms nodig om de (interessante) itemsets te bepalen

### Itemsets Efficiënte algoritmes
#### Efficiëntere algoritmen > Apriori
1. Bepaal een minimum support vb. 0,2
2. for (i=1; i < aantal mogelijke items; i++)
a. Bereken de supports van combinaties van i items. 
Doe dit enkel voor combinaties waarin geen van de geschrapte combinaties inzitten. 
Je weet namelijk dat de support nooit hoger zal zijn.
b. Schrap supports < minimum support

In [141]:
#In phyton
itemsets = apriori(transactionstable, min_support=0.2, use_colnames=True)
print(itemsets)

NameError: name 'transactionstable' is not defined

Pros:
 Een combinatie (AB) met een lage Support zal door uitbreiding van die combinatie (ABC, ABD, ...) nooit stijgen in Support!
 Apriori snijdt deze oninteressante combinaties automatisch weg (pruned).
 Door vanaf het begin combinaties die te weinig voorkomen (een lage Support hebben) te schrappen, worden veel scans van de transacties vermeden!
Cons:
 Hoewel efficiënter, vereist Apriori nog steeds voor elke te onderzoeken combinatie een scan van alle transacties.
 Bedrijven hebben nood aan het opstellen van associatieregels voor datasets met grote aantallen transacties en items. In dergelijke situaties duurt het Apriori algoritme zeer lang

#### FP-Growth
1. Tel de frequenties van alle items. Het algoritme doorloopt een eerste keer de transacties
2. Schrap de items met een support kleiner dan minimumsupport
3. Sorteer de items van groot naar klein volgens support

5. Bouw transactie per transactie een boomstructuur op. Het algoritme doorloopt een tweede keer alle transacties
A. Maak een eerste node. Dat is de Null node.
B. Neem de eerste transactie en sorteer de items uit die transactie volgens de volgorde uit stap 3
=> Cartridge, Printer, Ballpoint
C. Plaats de items in die volgorde in de boom.
> I. Als er nog geen subnode van het item bestaat dan voeg je die toe met een counter op 1. Aangezien dit de eerste transactie is betreft het hier sowieso nieuwe subnodes.
> II. Als het item al wel aan de null gekoppeld is dan verhoog je de counters van de subnodes die overeenkomen met je transactie. Andere voeg je de items toe via een nieuwe tak toe met counter op 1
D. Neem de volgende transactie, sorteer en voer de zelfde logica uit als in stap C.
Het eerste item (Cartridge) is reeds aan de null node gekoppeld. We verhogen daarom de counter op Cartridge met 1. Paper is nog niet gekoppeld aan die Cartridge
node. We voegen daarom een paper node toe met counter op 1.
E. Voor elke daaropvolgende transactie volgen we dezelfde logica.

5. Extraheer de supports uit de boomstructuur
• De boomstructuur bevat alle supports.
• De counter op een node geeft aan hoe vaak een combinatie met de bovenliggende nodes voorkomt.
Printer komt 3 keer voor met Cartridge.
• Het is ook mogelijk dat een bepaalde combinatie meerdere keren voorkomt.
“Printer, Ballpoint” komt 1 keer voor onder Cartridge en 1 keer voor onder Null. De combinatie “Printer,Ballpoint” komt dus 2 keer voor.
• Je dient ook subnodes te combineren met de parentnode van de parentnode Ballpoint komt 1 keer voor met Cartidge in combinatie met printer en 1 keer
in combinatie met Paper. “Cartridge, Ballpoint” komt dus 2 keer voor.
• De aantallen kunnen we gebruiken om te kijken welke combinaties voldoen aan de minimumsupport van 0.2 of in dit geval 2 van de 10 transacties:
Printer, Cartridge komt 3 keer voor
Paper, Cartridge komt 3 keer voor
Printer, Ballpoint komt 2 keer voor
Ballpoint, Cartridge komt 2 keer voor

In [142]:
# In python
itemsets = fpgrowth(transactionstable, min_support=0.2, use_colnames=True)
print(itemsets)

NameError: name 'transactionstable' is not defined

Pros: 
 Met het FP-growth algoritme moeten alle transacties slechts tweemaal doorlopen te worden om alle relevante supports te berekenen.
 Meer items geven enkel een lineaire groei
 Het werk kan ook verdeeld worden over verschillende computers. Deze manier van parallel werken maakt het mogelijk om dit algoritme op heel veel transacties uit te voeren.
Cons:
 Het FP-Growth algoritme is omslachtiger en moeilijker (handmatig) uit te voeren dan het Apriori algoritme.
 De uitvoering van het FP-Growth algoritme kan duur zijn.
 Wanneer de database groot is, past het algoritme mogelijk niet in het gedeelde geheugen.

### Association Rules bepalen
#### Identificeren > Confidence
Confidence: Hoe zeker zijn we er van dat item B in een transactie zit als we weten dat item A in een transactie zit?
Confidence A -> B = (Support A & B)/ Support A

#### Identificeren > Hoe regels vastleggen
Lift: in welke mate levert een bepaalde regel meer zekerheid op.
Lift A -> B = (Confidence A -> B)/ Support B

#### Verbanden met kansen
Support en Confidence zijn in feite kansen:
Support is de kans op gebeurtenis A (voorafgaande aan een verkoop):
Support(A) = #A/#Transacties = P(A)

Confidence is de voorwaardelijke kans op gebeurtenis B wanneer gebeurtenis A reeds heeft plaast gevonden (na een bepaalde koop):
Confidence A -> B = (Support A & B)/ Support A = P(B|A)

Wat met Lift?
Lift A -> B = (Confidence A -> B)/ Support B =P(B|A)/P(B)

Modeling > Hoe regels vastleggen
Als het aantal items beperkt is, kan je alle mogelijke itemsets opsommen. 
Zo niet, gebruik dan het apriori‐algoritme of het FP‐ Growth‐algoritme om het aantal itemsets te beperken.

1. Bereken alle mogelijke confidences...
a) ... voor itemsets met 2 item: Conf(Printer -> Paper)
b) ... voor itemsets met 3 items: Conf(Printer, Paper -> Cartridge)
c) ... voor itemsets met 4 items: Conf(Printer, Paper, Cartridge -> Ballpoint)
d) ...
2. Selecteer rules die je wil gebruiken
a) Regels met een confidence > x. Alle confidences groter dan 0,5 zetten we om naar een association rule
b) De n hoogste confidences: Als je moeilijk het niveau van de confidences kan inschatten 
Neem de regels met de hoogste confidences. Bol.com stelt andere items voor op basis van de top 3 confidences ten opzichte van het opgezochte item

In [143]:
rules = association_rules(itemsets, metric='confidence', min_threshold=0.4)
print(rules)

NameError: name 'itemsets' is not defined